## 1. 데이터 불러오기

### 필수 라이브러리

In [53]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split,GridSearchCV
from tqdm import tqdm

In [2]:
# 파일 경로
ROOT_DIR = "data"

y = pd.read_csv(os.path.join(ROOT_DIR, "train_y.csv"))

In [ ]:
# ROOT_DIR = "data"
# RANDOM_STATE = 110

# X_Dam = read_excel_file(os.path.join(ROOT_DIR, "Dam dispensing.xlsx"))

# X_AutoClave = read_excel_file(
#     os.path.join(ROOT_DIR, "Auto clave.xlsx")
# )

# X_Fill1 = read_excel_file(
#     os.path.join(ROOT_DIR, "Fill1 dispensing.xlsx")
# )

# X_Fill2 = read_excel_file(
#     os.path.join(ROOT_DIR, "Fill2 dispensing.xlsx")
# )

# y = pd.read_csv(os.path.join(ROOT_DIR, "train_y.csv"))

## X_Dam 분석

In [3]:
#Dam dispensing
file_name = 'Dam dispensing_header.csv'  # 원본 파일 이름
file_path = os.path.join(ROOT_DIR, file_name)
X_Dam = pd.read_csv(file_path, low_memory=True)

/var/folders/qj/004xgdtd15520sxppmd4bvvh0000gn/T/ipykernel_2970/3941712864.py:4: DtypeWarning: Columns (62,64) have mixed types. Specify dtype option on import or set low_memory=False.
  X_Dam = pd.read_csv(file_path, low_memory=True)


In [ ]:
type(X_Dam)

In [ ]:
X_Dam.info()

In [ ]:
X_Dam.describe()

In [ ]:
# pandas 출력 옵션 설정: 생략 없이 모든 열을 출력
# pd.set_option('display.max_rows', None)
X_Dam.isnull().sum()

In [4]:
# 결과를 저장할 빈 리스트 생성
results = []
# 각 열의 고유 값 개수를 저장할 빈 리스트 생성
unique_counts = []

# 각 열의 모든 값이 동일한지 확인
for column in X_Dam.columns:
    if X_Dam[column].nunique() == 1:
        print(f"All values in column '{column}' are the same.")
        results.append("same")
    else:
        print(f"Column '{column}' has different values.")
        results.append("different")
    
    unique_count = X_Dam[column].nunique()
    unique_counts.append(unique_count)

# 고유 값 개수를 데이터프레임의 마지막 행에 추가
X_Dam.loc['고유 개수'] = unique_counts
X_Dam.loc['결과'] = results
# 마지막 두 행 출력
last_two_rows = X_Dam.iloc[-2:]
print(last_two_rows)

All values in column 'Wip Line' are the same.
All values in column 'Process Desc.' are the same.
Column 'Equipment' has different values.
Column 'Model.Suffix' has different values.
Column 'Workorder' has different values.
Column 'LOT ID' has different values.
Column 'Set ID' has different values.
Column 'Box ID' has different values.
Column 'Collect Date' has different values.
Column 'Insp. Seq No.' has different values.
All values in column 'Insp Judge Code' are the same.
Column 'CURE END POSITION X_Collect Result' has different values.
Column 'CURE END POSITION X_Unit Time' has different values.
Column 'CURE END POSITION X_Judge Value' has different values.
Column 'CURE END POSITION Z_Collect Result' has different values.
Column 'CURE END POSITION Z_Unit Time' has different values.
Column 'CURE END POSITION Z_Judge Value' has different values.
Column 'CURE END POSITION Θ_Collect Result' has different values.
Column 'CURE END POSITION Θ_Unit Time' has different values.
Column 'CURE E

In [ ]:
# 마지막 행에서 'same' 값을 포함하는 열의 개수를 계산
last_row = X_Dam.iloc[-1]
same_count = (last_row == 'same').sum()

# 결과 출력
print(f"Number of columns containing 'same' in the last row: {same_count}")

In [5]:
import pandas as pd

# 마지막 행에서 'same' 값이 있는 열을 찾기
columns_to_exclude1 = X_Dam.iloc[-1][X_Dam.iloc[-1] == 'same'].index

# 마지막에서 두 번째 행에서 값이 0인 열을 찾기
columns_to_exclude2 = X_Dam.iloc[-2][X_Dam.iloc[-2] == 0].index

# 두 리스트를 합치고 중복을 제거
columns_to_exclude = list(set(columns_to_exclude1).union(set(columns_to_exclude2)))

# 해당 열을 제외한 새로운 데이터프레임 생성
X_Dam = X_Dam.drop(columns=columns_to_exclude)

print(X_Dam)

              Equipment Model.Suffix   Workorder                LOT ID  \
0      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000002   
1      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000003   
2      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000004   
3      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000005   
4      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000006   
...                 ...          ...         ...                   ...   
62476  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002687   
62477  Dam dispenser #1  AJX75334505  4F1XB738-1  OP753345054040002688   
62478  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002689   
고유 개수                 2            8         679                 59271   
결과            different    different   different             different   

                     Set ID         Collect Date Insp. Seq No.  \
0      OP753345013050000002  2023-05-04 08:57

In [6]:
# 두 열을 합치기
# NaN 값을 0으로 변환
X_Dam['WorkMode_Collect Result'] = X_Dam['WorkMode_Collect Result'].fillna(0)
X_Dam['Unnamed: 221'] = X_Dam['Unnamed: 221'].fillna(0)

X_Dam['WorkMode_Collect Result'] = X_Dam['WorkMode_Collect Result'] + X_Dam['Unnamed: 221']

# 'Unnamed: 221' 열 제거
X_Dam.drop('Unnamed: 221', axis=1, inplace=True)
print(X_Dam)

              Equipment Model.Suffix   Workorder                LOT ID  \
0      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000002   
1      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000003   
2      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000004   
3      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000005   
4      Dam dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000006   
...                 ...          ...         ...                   ...   
62476  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002687   
62477  Dam dispenser #1  AJX75334505  4F1XB738-1  OP753345054040002688   
62478  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002689   
고유 개수                 2            8         679                 59271   
결과            different    different   different             different   

                     Set ID         Collect Date Insp. Seq No.  \
0      OP753345013050000002  2023-05-04 08:57

In [ ]:
pd.set_option('display.max_rows', None)
X_Dam.isnull().sum()

In [7]:
X_Dam = X_Dam.iloc[:-2]
# 마지막 두 행 출력
last_two_rows = X_Dam.iloc[-2:]
print(last_two_rows)

              Equipment Model.Suffix   Workorder                LOT ID  \
62477  Dam dispenser #1  AJX75334505  4F1XB738-1  OP753345054040002688   
62478  Dam dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002689   

                     Set ID         Collect Date Insp. Seq No.  \
62477  OP753345054040002688  2024-04-28 18:33:51             1   
62478  OP753345054040002689  2024-04-28 18:34:57             1   

      CURE END POSITION X_Collect Result CURE END POSITION Z_Collect Result  \
62477                              240.0                                2.5   
62478                             1000.0                               12.5   

      CURE END POSITION Θ_Collect Result  ...  \
62477                                -90  ...   
62478                                 90  ...   

      Stage3 Circle3 Distance Speed_Collect Result  \
62477                                         5800   
62478                                         5800   

      Stage3 Circle4 Distance

## X_Fill1 분석

In [8]:
#Fill1 dispensing
file_name = 'Fill1 dispensing_header.csv'  # 원본 파일 이름
file_path = os.path.join(ROOT_DIR, file_name)
X_Fill1 = pd.read_csv(file_path, low_memory=False)

In [ ]:
type(X_Fill1)

In [ ]:
X_Fill1.info()

In [ ]:
X_Fill1.describe()

In [ ]:
# pandas 출력 옵션 설정: 생략 없이 모든 열을 출력
# pd.set_option('display.max_rows', None)
X_Fill1.isnull().sum()

In [9]:
# 결과를 저장할 빈 리스트 생성
results = []
# 각 열의 고유 값 개수를 저장할 빈 리스트 생성
unique_counts = []

# 각 열의 모든 값이 동일한지 확인
for column in X_Fill1.columns:
    if X_Fill1[column].nunique() == 1:
        print(f"All values in column '{column}' are the same.")
        results.append("same")
    else:
        print(f"Column '{column}' has different values.")
        results.append("different")
    
    unique_count = X_Fill1[column].nunique()
    unique_counts.append(unique_count)

# 고유 값 개수를 데이터프레임의 마지막 행에 추가
X_Fill1.loc['고유 개수'] = unique_counts
X_Fill1.loc['결과'] = results
# 마지막 두 행 출력
last_two_rows = X_Fill1.iloc[-2:]
print(last_two_rows)

All values in column 'Wip Line' are the same.
All values in column 'Process Desc.' are the same.
Column 'Equipment' has different values.
Column 'Model.Suffix' has different values.
Column 'Workorder' has different values.
Column 'LOT ID' has different values.
Column 'Set ID' has different values.
Column 'Box ID' has different values.
Column 'Collect Date' has different values.
Column 'Insp. Seq No.' has different values.
All values in column 'Insp Judge Code' are the same.
Column 'DISCHARGED SPEED OF RESIN_Collect Result' has different values.
Column 'DISCHARGED SPEED OF RESIN_Unit Time' has different values.
Column 'DISCHARGED SPEED OF RESIN_Judge Value' has different values.
Column 'DISCHARGED TIME OF RESIN(Stage1)_Collect Result' has different values.
Column 'DISCHARGED TIME OF RESIN(Stage1)_Unit Time' has different values.
Column 'DISCHARGED TIME OF RESIN(Stage1)_Judge Value' has different values.
Column 'DISCHARGED TIME OF RESIN(Stage2)_Collect Result' has different values.
Colum

In [ ]:
# 마지막 행에서 'same' 값을 포함하는 열의 개수를 계산
last_row = X_Fill1.iloc[-1]
same_count = (last_row == 'same').sum()

# 결과 출력
print(f"Number of columns containing 'same' in the last row: {same_count}")

In [10]:
import pandas as pd

# 마지막 행에서 'same' 값이 있는 열을 찾기
columns_to_exclude1 = X_Fill1.iloc[-1][X_Fill1.iloc[-1] == 'same'].index

# 마지막에서 두 번째 행에서 값이 0인 열을 찾기
columns_to_exclude2 = X_Fill1.iloc[-2][X_Fill1.iloc[-2] == 0].index

# 두 리스트를 합치고 중복을 제거
columns_to_exclude = list(set(columns_to_exclude1).union(set(columns_to_exclude2)))

# 해당 열을 제외한 새로운 데이터프레임 생성
X_Fill1 = X_Fill1.drop(columns=columns_to_exclude)

print(X_Fill1)

                Equipment Model.Suffix   Workorder                LOT ID  \
0      Fill1 dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000002   
1      Fill1 dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000003   
2      Fill1 dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000004   
3      Fill1 dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000005   
4      Fill1 dispenser #1  AJX75334501  3F1X5847-2  OP753345013050000006   
...                   ...          ...         ...                   ...   
61925  Fill1 dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002687   
61926  Fill1 dispenser #1  AJX75334505  4F1XB738-1  OP753345054040002688   
61927  Fill1 dispenser #2  AJX75334505  4F1XB738-1  OP753345054040002689   
고유 개수                   2            7         665                 58808   
결과              different    different   different             different   

                     Set ID         Collect Date Insp. Seq No.  \
0      OP753345013050

In [ ]:
pd.set_option('display.max_rows', None)
X_Fill1.isnull().sum()

In [11]:
X_Fill1 = X_Fill1.iloc[:-2]
# 마지막 두 행 출력
last_two_rows = X_Fill1.iloc[-1:]
last_two_rows


,Equipment,Model.Suffix,Workorder,LOT ID,Set ID,Collect Date,Insp. Seq No.,DISCHARGED SPEED OF RESIN_Collect Result,DISCHARGED TIME OF RESIN(Stage1)_Collect Result,DISCHARGED TIME OF RESIN(Stage2)_Collect Result,...,Head Clean Position Y_Collect Result,Head Clean Position Z_Collect Result,Head Purge Position X_Collect Result,Head Purge Position Y_Collect Result,Head Purge Position Z_Collect Result,Machine Tact time_Collect Result,PalletID_Collect Result,Production Qty_Collect Result,Receip No_Collect Result,WorkMode_Collect Result
61927,Fill1 dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002689,OP753345054040002689,2024-04-28 18:36:31,1,10.6,12.8,3.6,...,123.4,50.0,92.2,289,50,128.0,54.7,13,439,1


## X_Fill2 분석

In [12]:

#Fill2 dispensing
file_name = 'Fill2 dispensing_header.csv'  # 원본 파일 이름
file_path = os.path.join(ROOT_DIR, file_name)
X_Fill2 = pd.read_csv(file_path, low_memory=False)

In [ ]:
type(X_Fill2)

In [ ]:
X_Fill2.info()

In [ ]:
X_Fill2.describe()

In [ ]:
# pandas 출력 옵션 설정: 생략 없이 모든 열을 출력
# pd.set_option('display.max_rows', None)
X_Fill2.isnull().sum()

In [13]:
# 결과를 저장할 빈 리스트 생성
results = []
# 각 열의 고유 값 개수를 저장할 빈 리스트 생성
unique_counts = []

# 각 열의 모든 값이 동일한지 확인
for column in X_Fill2.columns:
    if X_Fill2[column].nunique() == 1:
        print(f"All values in column '{column}' are the same.")
        results.append("same")
    else:
        print(f"Column '{column}' has different values.")
        results.append("different")
    
    unique_count = X_Fill2[column].nunique()
    unique_counts.append(unique_count)

# 고유 값 개수를 데이터프레임의 마지막 행에 추가
X_Fill2.loc['고유 개수'] = unique_counts
X_Fill2.loc['결과'] = results
# 마지막 두 행 출력
last_two_rows = X_Fill2.iloc[-2:]
last_two_rows

All values in column 'Wip Line' are the same.
All values in column 'Process Desc.' are the same.
Column 'Equipment' has different values.
Column 'Model.Suffix' has different values.
Column 'Workorder' has different values.
Column 'LOT ID' has different values.
Column 'Set ID' has different values.
Column 'Box ID' has different values.
Column 'Collect Date' has different values.
Column 'Insp. Seq No.' has different values.
All values in column 'Insp Judge Code' are the same.
Column 'CURE END POSITION X_Collect Result' has different values.
Column 'CURE END POSITION X_Unit Time' has different values.
Column 'CURE END POSITION X_Judge Value' has different values.
Column 'CURE END POSITION Z_Collect Result' has different values.
Column 'CURE END POSITION Z_Unit Time' has different values.
Column 'CURE END POSITION Z_Judge Value' has different values.
All values in column 'CURE END POSITION Θ_Collect Result' are the same.
Column 'CURE END POSITION Θ_Unit Time' has different values.
Column '

,Wip Line,Process Desc.,Equipment,Model.Suffix,Workorder,LOT ID,Set ID,Box ID,Collect Date,Insp. Seq No.,...,Production Qty_Collect Result,Production Qty_Unit Time,Production Qty_Judge Value,Receip No_Collect Result,Receip No_Unit Time,Receip No_Judge Value,WorkMode_Collect Result,WorkMode_Unit Time,WorkMode_Judge Value,Unnamed: 131
고유 개수,1,1,2,8,679,59253,59253,0.0,62318,4,...,490,0.0,0.0,609,0.0,0.0,7,0.0,0.0,2.0
결과,same,same,different,different,different,different,different,different,different,different,...,different,different,different,different,different,different,different,different,different,different


In [ ]:
# 마지막 행에서 'same' 값을 포함하는 열의 개수를 계산
last_row = X_Fill2.iloc[-1]
same_count = (last_row == 'same').sum()

# 결과 출력
print(f"Number of columns containing 'same' in the last row: {same_count}")

In [14]:
import pandas as pd

# 마지막 행에서 'same' 값이 있는 열을 찾기
columns_to_exclude1 = X_Fill2.iloc[-1][X_Fill2.iloc[-1] == 'same'].index

# 마지막에서 두 번째 행에서 값이 0인 열을 찾기
columns_to_exclude2 = X_Fill2.iloc[-2][X_Fill2.iloc[-2] == 0].index

# 두 리스트를 합치고 중복을 제거
columns_to_exclude = list(set(columns_to_exclude1).union(set(columns_to_exclude2)))

# 해당 열을 제외한 새로운 데이터프레임 생성
X_Fill2 = X_Fill2.drop(columns=columns_to_exclude)

X_Fill2

,Equipment,Model.Suffix,Workorder,LOT ID,Set ID,Collect Date,Insp. Seq No.,CURE END POSITION X_Collect Result,CURE END POSITION Z_Collect Result,CURE SPEED_Collect Result,...,Head Clean Position Z_Collect Result,Head Purge Position X_Collect Result,Head Purge Position Y_Collect Result,Head Purge Position Z_Collect Result,Machine Tact time_Collect Result,PalletID_Collect Result,Production Qty_Collect Result,Receip No_Collect Result,WorkMode_Collect Result,Unnamed: 131
0,Fill2 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000002,OP753345013050000002,2023-05-04 09:00:48,1,240,32,55,...,91.8,270.0,50,85,17.8,7.0,1,1,0,NaN
1,Fill2 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000003,OP753345013050000003,2023-05-04 09:15:00,1,240,32,55,...,91.8,270.0,50,85,18.2,5.0,2,1,0,NaN
2,Fill2 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000004,OP753345013050000004,2023-05-04 09:17:04,1,240,32,55,...,91.8,270.0,50,85,18.4,6.0,3,1,0,NaN
3,Fill2 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000005,OP753345013050000005,2023-05-04 09:19:08,1,240,32,55,...,91.8,270.0,50,85,18.6,7.0,4,1,0,NaN
4,Fill2 dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000006,OP753345013050000006,2023-05-04 09:21:11,1,240,32,55,...,91.8,270.0,50,85,18.1,8.0,5,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62315,Fill2 dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002687,OP753345054040002687,2024-04-28 18:35:50,1,240,33,53,...,50.0,91.8,270,50,114.612,19.0,12,437,1,0.0
62316,Fill2 dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002688,OP753345054040002688,2024-04-28 18:36:34,1,240,33,53,...,50.0,91.8,270,50,114.612,18.8,6,438,1,0.0
62317,Fill2 dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002689,OP753345054040002689,2024-04-28 18:37:38,1,240,33,53,...,50.0,91.8,270,50,114.612,18.8,13,439,1,0.0
고유 개수,2,8,679,59253,59253,62318,4,2,3,13,...,2.0,2.0,2,2,46.0,169.0,490,609,7,2.0


In [ ]:
pd.set_option('display.max_rows', None)
X_Fill2.isnull().sum()

In [15]:
X_Fill2 = X_Fill2.iloc[:-2]
# 마지막 두 행 출력
last_two_rows = X_Fill2.iloc[-1:]
last_two_rows


,Equipment,Model.Suffix,Workorder,LOT ID,Set ID,Collect Date,Insp. Seq No.,CURE END POSITION X_Collect Result,CURE END POSITION Z_Collect Result,CURE SPEED_Collect Result,...,Head Clean Position Z_Collect Result,Head Purge Position X_Collect Result,Head Purge Position Y_Collect Result,Head Purge Position Z_Collect Result,Machine Tact time_Collect Result,PalletID_Collect Result,Production Qty_Collect Result,Receip No_Collect Result,WorkMode_Collect Result,Unnamed: 131
62317,Fill2 dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002689,OP753345054040002689,2024-04-28 18:37:38,1,240,33,53,...,50.0,91.8,270,50,114.612,18.8,13,439,1,0.0


## X_AutoClave 분석

In [16]:
#Auto clave
file_name = 'Auto clave_header.csv'  # 원본 파일 이름
file_path = os.path.join(ROOT_DIR, file_name)
X_AutoClave = pd.read_csv(file_path, low_memory=False)


In [ ]:
type(X_AutoClave)

In [ ]:
X_AutoClave.info()

In [ ]:
X_AutoClave.describe()

In [ ]:
# pandas 출력 옵션 설정: 생략 없이 모든 열을 출력
# pd.set_option('display.max_rows', None)
X_AutoClave.isnull().sum()

In [17]:
# 결과를 저장할 빈 리스트 생성
results = []
# 각 열의 고유 값 개수를 저장할 빈 리스트 생성
unique_counts = []

# 각 열의 모든 값이 동일한지 확인
for column in X_AutoClave.columns:
    if X_AutoClave[column].nunique() == 1:
        print(f"All values in column '{column}' are the same.")
        results.append("same")
    else:
        print(f"Column '{column}' has different values.")
        results.append("different")
    
    unique_count = X_AutoClave[column].nunique()
    unique_counts.append(unique_count)

# 고유 값 개수를 데이터프레임의 마지막 행에 추가
X_AutoClave.loc['고유 개수'] = unique_counts
X_AutoClave.loc['결과'] = results
# 마지막 두 행 출력
last_two_rows = X_AutoClave.iloc[-2:]
print(last_two_rows)

All values in column 'Wip Line' are the same.
All values in column 'Process Desc.' are the same.
All values in column 'Equipment' are the same.
Column 'Model.Suffix' has different values.
Column 'Workorder' has different values.
Column 'LOT ID' has different values.
Column 'Set ID' has different values.
Column 'Box ID' has different values.
Column 'Collect Date' has different values.
Column 'Insp. Seq No.' has different values.
All values in column 'Insp Judge Code' are the same.
Column '1st Pressure_Collect Result' has different values.
Column '1st Pressure_Unit Time' has different values.
All values in column '1st Pressure_Judge Value' are the same.
Column '2nd Pressure_Collect Result' has different values.
Column '2nd Pressure_Unit Time' has different values.
All values in column '2nd Pressure_Judge Value' are the same.
Column '3rd Pressure_Collect Result' has different values.
Column '3rd Pressure_Unit Time' has different values.
All values in column '3rd Pressure_Judge Value' are 

In [ ]:
# 마지막 행에서 'same' 값을 포함하는 열의 개수를 계산
last_row = X_AutoClave.iloc[-1]
same_count = (last_row == 'same').sum()

# 결과 출력
print(f"Number of columns containing 'same' in the last row: {same_count}")

In [18]:
import pandas as pd

# 마지막 행에서 'same' 값이 있는 열을 찾기
columns_to_exclude1 = X_AutoClave.iloc[-1][X_AutoClave.iloc[-1] == 'same'].index

# 마지막에서 두 번째 행에서 값이 0인 열을 찾기
columns_to_exclude2 = X_AutoClave.iloc[-2][X_AutoClave.iloc[-2] == 0].index

# 두 리스트를 합치고 중복을 제거
columns_to_exclude = list(set(columns_to_exclude1).union(set(columns_to_exclude2)))

# 해당 열을 제외한 새로운 데이터프레임 생성
X_AutoClave = X_AutoClave.drop(columns=columns_to_exclude)

X_AutoClave


,Model.Suffix,Workorder,LOT ID,Set ID,Collect Date,Insp. Seq No.,1st Pressure_Collect Result,1st Pressure_Unit Time,2nd Pressure_Collect Result,2nd Pressure_Unit Time,3rd Pressure_Collect Result,3rd Pressure_Unit Time,Chamber Temp._Collect Result,Chamber Temp._Unit Time,Chamber Temp._Judge Value
0,AJX75334501,3F1X5847-2,OP753345013050000002,OP753345013050000002,2023-05-04 09:29:00,1,0.312,300,0.504,120,0.503,90,56,510,OK
1,AJX75334501,3F1X5847-2,OP753345013050000003,OP753345013050000003,2023-05-04 09:49:12,1,0.312,300,0.502,120,0.501,90,50,510,OK
2,AJX75334501,3F1X5847-2,OP753345013050000004,OP753345013050000004,2023-05-04 09:49:12,1,0.312,300,0.502,120,0.501,90,50,510,OK
3,AJX75334501,3F1X5847-2,OP753345013050000005,OP753345013050000005,2023-05-04 09:49:12,1,0.312,300,0.502,120,0.501,90,50,510,OK
4,AJX75334501,3F1X5847-2,OP753345013050000006,OP753345013050000006,2023-05-04 09:49:12,1,0.312,300,0.502,120,0.501,90,50,510,OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61049,EAT65200901,4EPM0222-2,OP652009014040000068,OP652009014040000068,2024-04-20 11:52:26,1,0.31,120,0.309,120,0.5,120,36,360,OK
61050,EAT65200901,4EPM0222-2,OP652009014040000069,OP652009014040000069,2024-04-20 11:52:26,1,0.31,120,0.309,120,0.5,120,36,360,OK
61051,EAT65200901,4EPM0222-2,OP652009014040000070,OP652009014040000070,2024-04-20 11:42:24,1,0.312,120,0.311,120,0.498,120,36,360,OK
고유 개수,8,677,58317,58317,8766,4,28.0,11,69.0,10,74.0,11,28,25,2


In [ ]:
pd.set_option('display.max_rows', None)
X_AutoClave.isnull().sum()

In [19]:
X_AutoClave = X_AutoClave.iloc[:-2]

## 데이터 결합

In [20]:
# Rename columns
X_Dam.columns = [i + " - Dam" for i in X_Dam.columns]
X_AutoClave.columns = [i + " - AutoClave" for i in X_AutoClave.columns]
X_Fill1.columns = [i + " - Fill1" for i in X_Fill1.columns]
X_Fill2.columns = [i + " - Fill2" for i in X_Fill2.columns]

X_Dam = X_Dam.rename(columns={"Set ID - Dam": "Set ID"})
X_AutoClave = X_AutoClave.rename(columns={"Set ID - AutoClave": "Set ID"})
X_Fill1 = X_Fill1.rename(columns={"Set ID - Fill1": "Set ID"})
X_Fill2 = X_Fill2.rename(columns={"Set ID - Fill2": "Set ID"})

# Merge X
X = pd.merge(X_Dam, X_AutoClave, on="Set ID")
X = pd.merge(X, X_Fill1, on="Set ID")
X = pd.merge(X, X_Fill2, on="Set ID")
X = X.drop(X[X.duplicated(subset="Set ID")].index).reset_index(drop=True)
X

,Equipment - Dam,Model.Suffix - Dam,Workorder - Dam,LOT ID - Dam,Set ID,Collect Date - Dam,Insp. Seq No. - Dam,CURE END POSITION X_Collect Result - Dam,CURE END POSITION Z_Collect Result - Dam,CURE END POSITION Θ_Collect Result - Dam,...,Head Clean Position Z_Collect Result - Fill2,Head Purge Position X_Collect Result - Fill2,Head Purge Position Y_Collect Result - Fill2,Head Purge Position Z_Collect Result - Fill2,Machine Tact time_Collect Result - Fill2,PalletID_Collect Result - Fill2,Production Qty_Collect Result - Fill2,Receip No_Collect Result - Fill2,WorkMode_Collect Result - Fill2,Unnamed: 131 - Fill2
0,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000002,OP753345013050000002,2023-05-04 08:57:23,1,240.0,2.5,-90,...,91.8,270.0,50,85,17.8,7.0,1,1,0,NaN
1,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000003,OP753345013050000003,2023-05-04 09:11:35,1,240.0,2.5,-90,...,91.8,270.0,50,85,18.2,5.0,2,1,0,NaN
2,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000004,OP753345013050000004,2023-05-04 09:13:19,1,240.0,2.5,-90,...,91.8,270.0,50,85,18.4,6.0,3,1,0,NaN
3,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000005,OP753345013050000005,2023-05-04 09:15:24,1,240.0,2.5,-90,...,91.8,270.0,50,85,18.6,7.0,4,1,0,NaN
4,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000006,OP753345013050000006,2023-05-04 09:17:27,1,240.0,2.5,-90,...,91.8,270.0,50,85,18.1,8.0,5,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57862,Dam dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002685,OP753345054040002685,2024-04-28 18:30:42,1,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.4,11,435,1,0.0
57863,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002686,OP753345054040002686,2024-04-28 18:31:43,1,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.4,5,436,1,0.0
57864,Dam dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002687,OP753345054040002687,2024-04-28 18:32:49,1,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.0,12,437,1,0.0
57865,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002688,OP753345054040002688,2024-04-28 18:33:51,1,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,18.8,6,438,1,0.0


In [43]:
# Merge X and y
df_merged = pd.merge(X, y, "inner", on="Set ID")

# Drop columns with more than half of the values missing
drop_cols = []
for column in df_merged.columns:
    if (df_merged[column].notnull().sum() // 2) < df_merged[
        column
    ].isnull().sum():
        drop_cols.append(column)
df_merged = df_merged.drop(drop_cols, axis=1)

# Drop Lot ID
df_merged = df_merged.drop("LOT ID - Dam", axis=1)
df_merged

,Equipment - Dam,Model.Suffix - Dam,Workorder - Dam,Set ID,Collect Date - Dam,Insp. Seq No. - Dam,CURE END POSITION X_Collect Result - Dam,CURE END POSITION Z_Collect Result - Dam,CURE END POSITION Θ_Collect Result - Dam,CURE SPEED_Collect Result - Dam,...,Head Clean Position Z_Collect Result - Fill2,Head Purge Position X_Collect Result - Fill2,Head Purge Position Y_Collect Result - Fill2,Head Purge Position Z_Collect Result - Fill2,Machine Tact time_Collect Result - Fill2,PalletID_Collect Result - Fill2,Production Qty_Collect Result - Fill2,Receip No_Collect Result - Fill2,WorkMode_Collect Result - Fill2,target
0,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000003,2023-05-04 09:11:35,1,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.2,5.0,2,1,0,AbNormal
1,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000004,2023-05-04 09:13:19,1,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.4,6.0,3,1,0,AbNormal
2,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000007,2023-05-04 09:19:31,1,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.6,1.0,6,1,0,AbNormal
3,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000011,2023-05-04 11:19:55,1,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.2,4.0,18,1,0,AbNormal
4,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000024,2023-05-05 09:35:48,1,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.1,6.0,12,1,0,AbNormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002680,2024-04-28 18:25:20,1,240.0,2.5,-90,100,...,50.0,91.8,270,50,114.612,18.8,2,430,1,Normal
40502,Dam dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002681,2024-04-28 18:26:27,1,1000.0,12.5,90,100,...,50.0,91.8,270,50,114.612,19.0,9,431,1,Normal
40503,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002682,2024-04-28 18:27:27,1,240.0,2.5,-90,100,...,50.0,91.8,270,50,114.612,19.3,3,432,1,Normal
40504,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002684,2024-04-28 18:29:36,1,240.0,2.5,-90,100,...,50.0,91.8,270,50,114.612,19.6,4,434,1,Normal


In [44]:
RANDOM_STATE =110
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = df_merged[df_merged["target"] == "Normal"]
df_abnormal = df_merged[df_merged["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(
    n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE
)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
Name: count, dtype: int64

In [22]:
df_concat = df_concat.sort_values(by=["Collect Date - Dam"])
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.3,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)


def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(
        f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}"
        + f" ratio: {num_abnormal/num_normal}"
    )


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

NameError: name 'df_concat' is not defined

In [45]:
RANDOM_STATE =1004
df_merged = df_merged.sort_values(by=["Collect Date - Dam"])
df_train, df_val = train_test_split(
    df_merged,
    test_size=0.4,
    stratify=df_merged["target"],
    random_state=RANDOM_STATE,
)


def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(
        f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}"
        + f" ratio: {num_abnormal/num_normal}"
    )


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 22893, AbNormal: 1410 ratio: 0.06159087930808544
  Total: Normal: 15263, AbNormal: 940 ratio: 0.0615868440018345


In [54]:
# 모델 학습 및 하이퍼파라미터 튜닝
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = RandomForestClassifier(random_state=RANDOM_STATE)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           scoring='f1_weighted', cv=3, verbose=2, n_jobs=-1)
grid_search.fit(train_x, train_y)

best_model = grid_search.best_estimator_
print("Best parameters found: ", grid_search.best_params_)


Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   1.1s
[CV] END max_dep

In [55]:
model = RandomForestClassifier(
    n_estimators=100, 
    max_depth=20, 
    min_samples_split=10, 
    min_samples_leaf=1, 
    random_state=RANDOM_STATE
)

In [56]:
features = []

for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue

if "Set ID" in features:
    features.remove("Set ID")

train_x = df_train[features]
train_y = df_train["target"]

model.fit(train_x, train_y)

RandomForestClassifier(max_depth=20, min_samples_split=10, random_state=1004)

In [57]:
df_test_y = pd.read_csv(os.path.join("submission.csv"))

In [58]:
df_test = pd.merge(X, df_test_y, "inner", on="Set ID")
df_test_x = df_test[features]

for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

In [59]:
test_pred = model.predict(df_test_x)
test_pred

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

In [60]:
from sklearn.metrics import f1_score

# Assuming 'target' is the label column in df_val
val_y_true = df_val['target']
val_pred = model.predict(df_val[features])  # 예측 값

# Calculate F1 score
f1 = f1_score(val_y_true, val_pred, average='weighted')
print(f"F1 Score: {f1}")

F1 Score: 0.9196376605494054


In [63]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)